<a href="https://colab.research.google.com/github/perceus32/scheduling-tool/blob/main/Scheduling_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from enum import Enum

In [ ]:
qualcomm_df = pd.read_csv('/content/gdrive/MyDrive/PU_Scheduling_Tool/Qualcomm - Sheet1.csv')
micron_df = pd.read_csv('/content/gdrive/MyDrive/PU_Scheduling_Tool/Micron - Sheet1.csv')
ti_df = pd.read_csv('/content/gdrive/MyDrive/PU_Scheduling_Tool/TI - Sheet1.csv')

#qualcomm_df.head()
#micron_df.head()

# Merging the dataframes and sorting them on the basis of priority

In [ ]:
qualcomm_df.rename(columns = {'Status':'Qualcomm_Status'}, inplace = True)
micron_df.rename(columns = {'Status':'Micron_Status'}, inplace = True)
micron_df.rename(columns = {'Email':'Email ID'}, inplace = True)
ti_df.rename(columns = {'Status':'TI_Status'}, inplace = True)


#qualcomm_df
#micron_df
#ti_df.head()

In [ ]:
#df_intermediate = pd.merge(qualcomm_df, micron_df, on=["Name","Email ID"], how="outer")
df_intermediate = pd.merge(micron_df, ti_df, on=["Name","Email ID"], how="outer")
 
#df_final = pd.merge(ti_df, df_intermediate, on=["Name","Email ID"], how="outer")
df_final = pd.merge(qualcomm_df, df_intermediate, on=["Name","Email ID"], how="outer")

df_final.drop_duplicates(subset=['Name','Email ID'], keep='first', inplace=True, ignore_index=True)
#df_final.head()

In [ ]:
df_final['Micron_Status'] = df_final['Micron_Status'].fillna(0)
df_final['TI_Status'] = df_final['TI_Status'].fillna(0)
df_final['Qualcomm_Status'] = df_final['Qualcomm_Status'].fillna(0)
#df_final.head()

In [ ]:
df_final['PR'] = df_final['Micron_Status']+df_final['TI_Status']+df_final['Qualcomm_Status']
#df_final.head()

In [ ]:
df_final = df_final.sort_values(
        by="PR",
        ascending=False
)
#df_final

In [ ]:
df_final.reset_index(drop=True, inplace=True)
#df_final.head()

# Algorithm

In [ ]:
timeslots_companies = [8,8,8] #This will be taken in the input
#starting_timeslot = min(timeslots_companies)

panel_no_companies = [5, 6, 8] #input
no_of_companies = 3 #input



In [ ]:
from pandas._libs import index
matrix_df = pd.DataFrame()
matrix_df = pd.DataFrame(index = ['0', '1', '2', '3', '4', '5', '6', '7'], columns = ['0', '1', '2', '3', '4', '5', '6', '7'])
#matrix_df
#for i in range(matrix_df.shape[0]): #iterate over rows
    #for j in range(matrix_df.shape[1]): #iterate over columns
        #pair = (i, j)
       # matrix_df.at[i, j] = pair #get cell value


# Algorithm working part

In [ ]:
possible_combinations = []
time_slots_companies = [0,0,0]
panel_no_companies = [17, 10, 10] #input
slots_companies = [8,8,8]
no_of_companies = 3 #input

timeslots = {
    "9:00 - 9:45" : 0,
    "9:45 - 10:30" : 1,
    "10:30 - 11:15" : 2,
    "11:15 - 12:00" : 3,
    "12:00 - 12:45" : 4,
    "12:45 - 1:30" : 5,
    "1:30 - 2:15" : 6,
    "2:15 - 3:00" : 7,
} #8 timeslots

for k in range(0,no_of_companies):
  (x,y) = (time_slots_companies[k], panel_no_companies[k])
  possible_combinations.append([[[n,m,0] for m in range(y)] for n in range(x,x+slots_companies[k])])
#possible_combinations

In [ ]:
df_final['qualcomm_ts'] = np.nan
df_final['qualcomm_panel'] = np.nan

df_final['micron_ts'] = np.nan
df_final['micron_panel'] = np.nan

df_final['ti_ts'] = np.nan
df_final['ti_panel'] = np.nan

#df_final

In [ ]:
# Iterating through all the rows in the dataframe
for index, row in df_final.iterrows():
  # Non_iter represents the timeslot in which the candidate is busy
  non_iter = []

  # k represents the company index
  k=0

  # To check whether candidate has been shortlisted for Qualcomm
  if row['Qualcomm_Status'] != 0:
    breakout_flag = False
    for i in range(len(possible_combinations[k])):
  
      # Check whether candidate is busy during this time period, if yes
      # continue to the next iteration
      if i in non_iter:
        continue
      for j in range(len(possible_combinations[k][0])):
        
        # Check for the first panel which is free
        if possible_combinations[k][i][j][2]==0:
          # Allot the timeslot and panel no
          df_final.at[index,'qualcomm_ts'] = possible_combinations[k][i][j][0]
          df_final.at[index,'qualcomm_panel']= possible_combinations[k][i][j][1]

          # Change the status of the timeslot and panel no combination
          possible_combinations[k][i][j][2] = 1

          # Make this timeslot unavailble for the next companies for the candidate
          non_iter.append(possible_combinations[k][i][j][0])

          # To break out from the outer loop
          breakout_flag = True
          # To break out from the inner loop
          break
      if breakout_flag:
        break
    if row['qualcomm_ts']==np.nan:
      print("Scheduler not working!")
  k=k+1
  if row['Micron_Status'] != 0:
    breakout_flag = False
    for i in range(len(possible_combinations[k])):
  
      # Check whether candidate is busy during this time period, if yes
      # continue to the next iteration
      if i in non_iter:
        continue
      for j in range(len(possible_combinations[k][0])):
        
        # Check for the first panel which is free
        if possible_combinations[k][i][j][2]==0:
          # Allot the timeslot and panel no
          df_final.at[index,'micron_ts'] = possible_combinations[k][i][j][0]
          df_final.at[index,'micron_panel']= possible_combinations[k][i][j][1]

          # Change the status of the timeslot and panel no combination
          possible_combinations[k][i][j][2] = 1

          # Make this timeslot unavailble for the next companies for the candidate
          non_iter.append(possible_combinations[k][i][j][0])

          # To break out from the outer loop
          breakout_flag = True
          # To break out from the inner loop
          break
      if breakout_flag:
        break
    if row['micron_ts']==np.nan:
      print("Scheduler not working!")
  k=k+1
  if row['TI_Status'] != 0:
    breakout_flag = False
    for i in range(len(possible_combinations[k])):
  
      # Check whether candidate is busy during this time period, if yes
      # continue to the next iteration
      if i in non_iter:
        continue
      for j in range(len(possible_combinations[k][0])):
        
        # Check for the first panel which is free
        if possible_combinations[k][i][j][2]==0:
          # Allot the timeslot and panel no
          df_final.at[index,'ti_ts'] = possible_combinations[k][i][j][0]
          df_final.at[index,'ti_panel']= possible_combinations[k][i][j][1]

          # Change the status of the timeslot and panel no combination
          possible_combinations[k][i][j][2] = 1

          # Make this timeslot unavailble for the next companies for the candidate
          non_iter.append(possible_combinations[k][i][j][0])

          # To break out from the outer loop
          breakout_flag = True
          # To break out from the inner loop
          break
      if breakout_flag:
        break
    if row['ti_ts']==np.nan:
      print("Scheduler not working!")

In [ ]:
df_final

,Name,Email ID,Qualcomm_Status,Micron_Status,TI_Status,PR,qualcomm_ts,qualcomm_panel,micron_ts,micron_panel,ti_ts,ti_panel
0,Arnav Sachin Pawar,h20210343@pilani.bits-pilani.ac.in,1.0,1.0,1.0,3.0,0.0,0.0,1.0,0.0,2.0,0.0
1,Pratik Jain,h20210241@pilani.bits-pilani.ac.in,1.0,1.0,1.0,3.0,0.0,1.0,1.0,1.0,2.0,1.0
2,Arpit Mathur,h20210236@pilani.bits-pilani.ac.in,1.0,1.0,1.0,3.0,0.0,2.0,1.0,2.0,2.0,2.0
3,Abhishek Verma,h20210214@pilani.bits-pilani.ac.in,1.0,1.0,1.0,3.0,0.0,3.0,1.0,3.0,2.0,3.0
4,Vishal Bhanudas Chekale,h20210157@pilani.bits-pilani.ac.in,1.0,1.0,1.0,3.0,0.0,4.0,1.0,4.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
113,Akshay Gola,h20210250@pilani.bits-pilani.ac.in,1.0,0.0,0.0,1.0,6.0,3.0,NaN,NaN,NaN,NaN
114,Saachi Jain,f20180197@pilani.bits-pilani.ac.in,1.0,0.0,0.0,1.0,6.0,4.0,NaN,NaN,NaN,NaN
115,Ashish Pathak,f20180048@pilani.bits-pilani.ac.in,1.0,0.0,0.0,1.0,6.0,5.0,NaN,NaN,NaN,NaN
116,Amisha Mishra,f20180825@pilani.bits-pilani.ac.in,1.0,0.0,0.0,1.0,6.0,6.0,NaN,NaN,NaN,NaN


In [ ]:
df_final.loc[df_final['Qualcomm_Status'] == 1, 'Qualcomm_Status'] = 'Shortlisted'
df_final.loc[df_final['Micron_Status'] == 1, 'Micron_Status'] = 'Shortlisted'
df_final.loc[df_final['TI_Status'] == 1, 'TI_Status'] = 'Shortlisted'
df_final = df_final.drop(columns=['PR'])

In [ ]:
times = ['9:00-9:45', '9:45-10:30', '10:30-11:15', '11:15-12:00', '12:00-12:45', '12:45-1:30', '1:30-2:15', '2:15-3:00']

for k in range(0,no_of_companies):
  (x,y) = (time_slots_companies[k], panel_no_companies[k])
  for i in range(x, x+slots_companies[k]):
    df_final.loc[df_final['qualcomm_ts'] == i, 'qualcomm_ts'] = times[i]
    df_final.loc[df_final['micron_ts'] == i, 'micron_ts'] = times[i]
    df_final.loc[df_final['ti_ts'] == i, 'ti_ts'] = times[i]

df_final.head()

,Name,Email ID,Qualcomm_Status,Micron_Status,TI_Status,qualcomm_ts,qualcomm_panel,micron_ts,micron_panel,ti_ts,ti_panel
0,Arnav Sachin Pawar,h20210343@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,0.0,9:45-10:30,0.0,10:30-11:15,0.0
1,Pratik Jain,h20210241@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,1.0,9:45-10:30,1.0,10:30-11:15,1.0
2,Arpit Mathur,h20210236@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,2.0,9:45-10:30,2.0,10:30-11:15,2.0
3,Abhishek Verma,h20210214@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,3.0,9:45-10:30,3.0,10:30-11:15,3.0
4,Vishal Bhanudas Chekale,h20210157@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,4.0,9:45-10:30,4.0,10:30-11:15,4.0


In [ ]:
def increment_func(num):
  return num+1

df_final['qualcomm_panel']  = df_final['qualcomm_panel'].apply(increment_func)
df_final['micron_panel']  = df_final['micron_panel'].apply(increment_func)
df_final['ti_panel']  = df_final['ti_panel'].apply(increment_func)

df_final.head()

,Name,Email ID,Qualcomm_Status,Micron_Status,TI_Status,qualcomm_ts,qualcomm_panel,micron_ts,micron_panel,ti_ts,ti_panel
0,Arnav Sachin Pawar,h20210343@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,1.0,9:45-10:30,1.0,10:30-11:15,1.0
1,Pratik Jain,h20210241@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,2.0,9:45-10:30,2.0,10:30-11:15,2.0
2,Arpit Mathur,h20210236@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,3.0,9:45-10:30,3.0,10:30-11:15,3.0
3,Abhishek Verma,h20210214@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,4.0,9:45-10:30,4.0,10:30-11:15,4.0
4,Vishal Bhanudas Chekale,h20210157@pilani.bits-pilani.ac.in,Shortlisted,Shortlisted,Shortlisted,9:00-9:45,5.0,9:45-10:30,5.0,10:30-11:15,5.0


In [ ]:
def convert_na(val):
  if val==0:
    return np.nan
  else:
    return val

df_final['Qualcomm_Status'] = df_final['Qualcomm_Status'].apply(convert_na)
df_final['Micron_Status'] = df_final['Micron_Status'].apply(convert_na)
df_final['TI_Status'] = df_final['TI_Status'].apply(convert_na)

In [ ]:
df_final.to_csv('final_ET_schedule.csv')

In [ ]:
!cp final_ET_schedule.csv "/content/gdrive/MyDrive/PU_Scheduling_Tool"

In [ ]:
df_final.to_excel("final_ET_schedule.xlsx")

In [ ]:
!cp final_ET_schedule.xlsx "/content/gdrive/MyDrive/PU_Scheduling_Tool"

In [ ]:

df_qualcomm = df_final.drop(columns=['micron_ts', 'micron_panel', 'ti_ts', 'ti_panel'])
df_micron = df_final.drop(columns=['qualcomm_ts', 'qualcomm_panel', 'ti_ts', 'ti_panel'])
df_ti = df_final.drop(columns=['micron_ts', 'micron_panel', 'qualcomm_ts', 'qualcomm_panel'])

df_qualcomm.to_csv('qualcomm.csv')
df_micron.to_csv('micron.csv')
df_ti.to_csv('ti.csv')
